# Analyze data with Apache Spark - R version
R code by Antti Rask

Original Python version: https://microsoftlearning.github.io/mslearn-fabric/Instructions/Labs/02-analyze-spark.html

**Note!** This notebook was made primarily to be run inside Microsoft Fabric / Azure Synapse Data Engineering. You might have to make changes if you decide to open it in VS Code, for instance.

Apache Spark is an open source engine for distributed data processing, and is widely used to explore, process, and analyze huge volumes of data in data lake storage. Spark is available as a processing option in many data platform products, including Azure HDInsight, Azure Databricks, Azure Synapse Analytics, and Microsoft Fabric. One of the benefits of Spark is support for a wide range of programming languages, including Java, Scala, Python, and SQL; making Spark a very flexible solution for data processing workloads including data cleansing and manipulation, statistical analysis and machine learning, and data analytics and visualization.

This lab will take approximately **45** minutes to complete.

**Note:** You need a [Microsoft Fabric trial](https://learn.microsoft.com/fabric/get-started/fabric-trial) to complete this exercise.

## Create a workspace
Before working with data in Fabric, create a workspace with the Fabric trial enabled.

1. On the Microsoft Fabric home page at https://app.fabric.microsoft.com, select **Synapse Data Engineering**.
2. In the menu bar on the left, select **Workspaces** (the icon looks similar to 🗇).
3. Create a new workspace with a name of your choice, selecting a licensing mode in the **Advanced** section that includes Fabric capacity (_Trial, Premium, or Fabric_).
4. When your new workspace opens, it should be empty.

## Create a lakehouse and upload files
Now that you have a workspace, it’s time to create a data lakehouse for the data files you’re going to analyze.

1. In the **Synapse Data Engineering** home page, create a new **Lakehouse** with a name of your choice.

After a minute or so, a new empty lakehouse will be created. You need to ingest some data into the data lakehouse for analysis. There are multiple ways to do this, but in this exercise you’ll simply download and extract a folder of text files your local computer (or lab VM if applicable) and then upload them to your lakehouse.

2. Download and extract the data files for this exercise from https://github.com/MicrosoftLearning/dp-data/raw/main/orders.zip.
3. After extracting the zipped archive, verify that you have a folder named **orders** that contains CSV files named **2019.csv**, **2020.csv**, and **2021.csv**.
4. Return to the web browser tab containing your lakehouse, and in the **…** menu for the **Files** folder in the **Explorer** pane, select **Upload** and **Upload folder**, and then upload the **orders** folder from your local computer (or lab VM if applicable) to the lakehouse.
5. After the files have been uploaded, expand **Files** and select the **orders** folder; and verify that the CSV files have been uploaded.

## Create a notebook
To work with data in Apache Spark, you can create a notebook. Notebooks provide an interactive environment in which you can write and run code (in multiple languages), and add notes to document it.

1. On the Home page while viewing the contents of the **orders** folder in your datalake, in the **Open notebook** menu, select **New notebook**.

After a few seconds, a new notebook containing a single cell will open. Notebooks are made up of one or more cells that can contain code or markdown (formatted text).

2. Select the first cell (which is currently a code cell), and then in the dynamic tool bar at its top-right, use the **M↓** button to convert the cell to a markdown cell.

When the cell changes to a markdown cell, the text it contains is rendered.

3. Use the **🖉 (Edit)** button to switch the cell to editing mode, then modify the markdown as follows:

```
# Sales order data exploration

Use the code in this notebook to explore sales order data.
```
4. Click anywhere in the notebook outside of the cell to stop editing it and see the rendered markdown.

# Sales order data exploration

Use the code in this notebook to explore sales order data.

## Housekeeping

We'll get to the actual code soon, but first some housekeeping: installing and loading packages. And sorting out possible function conflicts.

**Note:** Spark supports multiple coding languages, including Scala, Java, and R. In this exercise, we’ll use RSpark, which is a Spark-optimized variant of R. Although the heavy lifting will be done by [sparklyr](https://spark.posit.co/), a tidyverse friendly R interface to Apache Spark.

### Installing and loading needed R packages

In [24]:
# Install packages
install.packages(
  c(
    "conflicted",
    "dplyr",
    "ggpie",
    "ggplot2",
    "lubridate",
    "patchwork",
    "sparklyr"
  )
)

StatementMeta(, 1b31353d-3791-4ef0-8beb-6d0eee069bdb, 26, Finished, Available)

In [ ]:
# Load packages
library(conflicted) # Handling function conflicts between packages
library(dplyr)      # Data wrangling
library(ggpie)      # Pie charts
library(ggplot2)    # Making charts in general
library(lubridate)  # Time wrangling
library(patchwork)  # Arranging charts
library(sparklyr)   # Using Spark with R

StatementMeta(, , , Cancelled, )

### Handling function conflicts between packages using the [{conflicted}](https://conflicted.r-lib.org/) package

In [ ]:
# Handling conflicts
conflicts_prefer(
    dplyr::arrange,
    dplyr::collect,
    dplyr::count,
    dplyr::distinct,
    dplyr::filter,
    dplyr::group_by,
    dplyr::mutate,
    dplyr::summarize,
    lubridate::year,
    sparklyr::select
)

StatementMeta(, , , Cancelled, )

### Setting up a Spark connection with {sparklyr}

In [ ]:
# Setting up sparklyr
spark_version <- sparkR.version()
config        <- spark_config()
sc            <- spark_connect(
    master  = "yarn",
    method  = "synapse",
    version = spark_version,
    config  = config
)

StatementMeta(, , , Cancelled, )

## Load data into a dataframe
Now you’re ready to run code that loads the data into a dataframe. Dataframes in Spark are similar to dataframes in R, and provide a common structure for working with data in rows and columns.

1. With the notebook visible, in the **Explorer** pane, expand **Lakehouses** and then expand the **Files** list for your lakehouse and select the **orders** folder so that the CSV files are listed next to the notebook editor.

2. In the … menu for **2019.csv**, select **Load data** > **Spark**. A new code cell containing the following code should be added to the notebook.

**Tip:** You can hide the Lakehouse explorer panes on the left by using their « icons. Doing so will help you focus on the notebook.

3. Use the **▷ Run cell** button on the left of the cell to run it.

**Note:** Since this is the first time you’ve run any Spark code, a Spark session must be started. This means that the first run in the session can take a minute or so to complete. Subsequent runs will be quicker.

4. When the cell command has completed, review the output below the cell.

The output shows the rows and columns of data from the 2019.csv file. However, note that the column headers don’t look right. The default code used to load the data into a dataframe assumes that the CSV file includes the column names in the first row, but in this case the CSV file just includes the data with no header information.

In [ ]:
df <- spark_read_csv(
    sc,
    path = "Files/orders/2019.csv"
    )
# df now is a Spark DataFrame containing CSV data from "Files/orders/2019.csv".
df

StatementMeta(, , , Cancelled, )

5. Modify the code to set the **header** option to **false** as follows:
6. Re-run the cell and review the output.

In [ ]:
df <- spark_read_csv(
    sc,
    path   = "Files/orders/2019.csv",
    header = FALSE
    )
# df now is a Spark DataFrame containing CSV data from "Files/orders/2019.csv".
df

StatementMeta(, , , Cancelled, )

Now the dataframe correctly includes first row as data values, but the column names are auto-generated and not very helpful. To make sense of the data, you need to explicitly define the correct schema and data type for the data values in the file.

7. Modify the code as follows to define a schema and apply it when loading the data:

8. Run the modified cell and review the output.

In [7]:
.columns <- list(
    SalesOrderNumber     = "string",
    SalesOrderLineNumber = "integer",
    OrderDate            = "date",
    CustomerName         = "string",
    Email                = "string",
    Item                 = "string",
    Quantity             = "integer",
    UnitPrice            = "double",
    Tax                  = "double"
)

df <- spark_read_csv(
    sc,
    path    = "Files/orders/2019.csv",
    header  = FALSE,
    columns = .columns
    )

df

StatementMeta(, 1b31353d-3791-4ef0-8beb-6d0eee069bdb, 9, Finished, Available)

# Source: spark<2019_eaaae494_e128_4be1_b852_4889e1000581> [?? x 9]
   SalesOrderNumber SalesOrderLineNumber OrderDate  CustomerName     Email Item 
   <chr>                           <int> <date>     <chr>            <chr> <chr>
 1 SO43701                             1 2019-07-01 Christy Zhu      chri… Moun…
 2 SO43704                             1 2019-07-01 Julio Ruiz       juli… Moun…
 3 SO43705                             1 2019-07-01 Curtis Lu        curt… Moun…
 4 SO43700                             1 2019-07-01 Ruben Prasad     rube… Road…
 5 SO43703                             1 2019-07-01 Albert Alvarez   albe… Road…
 6 SO43697                             1 2019-07-01 Cole Watson      cole… Road…
 7 SO43699                             1 2019-07-01 Sydney Wright    sydn… Moun…
 8 SO43702                             1 2019-07-01 Colin Anand      coli… Road…
 9 SO43698                             1 2019-07-01 Rachael Martinez rach… Moun…
10 SO43707                             1 

Now the dataframe includes the correct column names (in addition to the **Index**, which is a built-in column in all dataframes based on the ordinal position of each row). The data types of the columns are specified using a standard set of types defined in the Spark SQL library, which were imported at the beginning of the cell.

9. The dataframe includes only the data from the **2019.csv** file. Modify the code so that the file path uses a * wildcard to read the sales order data from all of the files in the **orders** folder:

10. Run the modified code cell and review the output, which should now include sales for 2019, 2020, and 2021.

**Note:** Only a subset of the rows is displayed, so you may not be able to see examples from all years.

In [8]:
df <- spark_read_csv(
    sc,
    path    = "Files/orders/*.csv",
    columns = .columns
    )

df

StatementMeta(, 1b31353d-3791-4ef0-8beb-6d0eee069bdb, 10, Finished, Available)

# Source: spark<_0ebe3a82_09c0_4530_b866_c18927eb8be9> [?? x 9]
   SalesOrderNumber SalesOrderLineNumber OrderDate  CustomerName     Email Item 
   <chr>                           <int> <date>     <chr>            <chr> <chr>
 1 SO43701                             1 2019-07-01 Christy Zhu      chri… Moun…
 2 SO43704                             1 2019-07-01 Julio Ruiz       juli… Moun…
 3 SO43705                             1 2019-07-01 Curtis Lu        curt… Moun…
 4 SO43700                             1 2019-07-01 Ruben Prasad     rube… Road…
 5 SO43703                             1 2019-07-01 Albert Alvarez   albe… Road…
 6 SO43697                             1 2019-07-01 Cole Watson      cole… Road…
 7 SO43699                             1 2019-07-01 Sydney Wright    sydn… Moun…
 8 SO43702                             1 2019-07-01 Colin Anand      coli… Road…
 9 SO43698                             1 2019-07-01 Rachael Martinez rach… Moun…
10 SO43707                             1 2019

## Explore data in a dataframe
The dataframe object includes a wide range of functions that you can use to filter, group, and otherwise manipulate the data it contains.

### Filter a dataframe
1. Add a new code cell by using the **+ Code** link that appears when you move the mouse under the left side of the current cell’s output (or on the menu bar, on the **Edit** tab, select **+ Add code cell**). Then enter the following code in it.

2. Run the new code cell, and review the results.

In [9]:
customers <- df %>%
    select(CustomerName, Email)

customers %>%
    count()

StatementMeta(, 1b31353d-3791-4ef0-8beb-6d0eee069bdb, 11, Finished, Available)

# Source: spark<?> [?? x 1]
      n
  <dbl>
1 65433

In [10]:
customers %>%
    distinct() %>%
    count()

StatementMeta(, 1b31353d-3791-4ef0-8beb-6d0eee069bdb, 12, Finished, Available)

# Source: spark<?> [?? x 1]
      n
  <dbl>
1 12427
Warning messages:
1: ORDER BY is ignored in subqueries without LIMIT
ℹ Do you need to move arrange() later in the pipeline or use window_order() instead? 
2: ORDER BY is ignored in subqueries without LIMIT
ℹ Do you need to move arrange() later in the pipeline or use window_order() instead? 
3: ORDER BY is ignored in subqueries without LIMIT
ℹ Do you need to move arrange() later in the pipeline or use window_order() instead? 

In [11]:
customers %>%
    distinct()

StatementMeta(, 1b31353d-3791-4ef0-8beb-6d0eee069bdb, 13, Finished, Available)

# Source: spark<?> [?? x 2]
   CustomerName  Email                      
   <chr>         <chr>                      
 1 Aaron Adams   aaron48@adventure-works.com
 2 Aaron Allen   aaron55@adventure-works.com
 3 Aaron Bryant  aaron17@adventure-works.com
 4 Aaron Chen    aaron25@adventure-works.com
 5 Aaron Collins aaron35@adventure-works.com
 6 Aaron Diaz    aaron21@adventure-works.com
 7 Aaron Edwards aaron34@adventure-works.com
 8 Aaron Evans   aaron33@adventure-works.com
 9 Aaron Flores  aaron11@adventure-works.com
10 Aaron Foster  aaron15@adventure-works.com
# ℹ more rows

Observe the following details:
- When you perform an operation on a dataframe, the result is a new dataframe (in this case, a new **customers** dataframe is created by selecting a specific subset of columns from the **df** dataframe)
- The **select()** function is the preferred [tidyverse](https://www.tidyverse.org/) way of defining a subset of columns
- [{dplyr}](https://dplyr.tidyverse.org/) provides functions such as **count()** and **distinct()** that can be used to summarize and filter the data the dataframe contains

3. Modify the code as follows:

4. Run the modified code to view the customers who have purchased the Road-250 Red, 52 product. Note that you can “chain” multiple functions together using the [{magrittr}](https://magrittr.tidyverse.org/) pipe (included with **{dplyr}**)so that the output of one function becomes the input for the next - in this case, the **filter()** function is is used to apply filtering criteria and is the source dataframe for the **select()** function that creates the **customers** dataframe.

In [12]:
customers <- df %>%
    filter(Item == "Road-250 Red, 52") %>%
    select(CustomerName, Email)

customers %>%
    count()

StatementMeta(, 1b31353d-3791-4ef0-8beb-6d0eee069bdb, 14, Finished, Available)

# Source: spark<?> [?? x 1]
      n
  <dbl>
1   266

In [13]:
customers %>%
    distinct() %>%
    count()

StatementMeta(, 1b31353d-3791-4ef0-8beb-6d0eee069bdb, 15, Finished, Available)

# Source: spark<?> [?? x 1]
      n
  <dbl>
1   133
Warning messages:
1: ORDER BY is ignored in subqueries without LIMIT
ℹ Do you need to move arrange() later in the pipeline or use window_order() instead? 
2: ORDER BY is ignored in subqueries without LIMIT
ℹ Do you need to move arrange() later in the pipeline or use window_order() instead? 
3: ORDER BY is ignored in subqueries without LIMIT
ℹ Do you need to move arrange() later in the pipeline or use window_order() instead? 

In [14]:
customers %>%
    distinct()

StatementMeta(, 1b31353d-3791-4ef0-8beb-6d0eee069bdb, 16, Finished, Available)

# Source: spark<?> [?? x 2]
   CustomerName     Email                           
   <chr>            <chr>                           
 1 Adam Jai         adam28@adventure-works.com      
 2 Adriana Sai      adriana6@adventure-works.com    
 3 Albert Navarro   albert12@adventure-works.com    
 4 Alejandro Raji   alejandro46@adventure-works.com 
 5 Alexander Walker alexander10@adventure-works.com 
 6 Alexandria Gray  alexandria29@adventure-works.com
 7 Alexis Long      alexis32@adventure-works.com    
 8 Andre Patel      andre3@adventure-works.com      
 9 Andrew Robinson  andrew25@adventure-works.com    
10 Ann Madan        ann13@adventure-works.com       
# ℹ more rows

### Aggregate and group data in a dataframe
1. Add a new code cell to the notebook, and enter the following code in it:

2. Run the code cell you added, and note that the results show the sum of order quantities grouped by product. The **group_by()** function groups the rows by Item, and the subsequent **sum()** function is applied inside the **summarize()** function to the **Quantity** column to create a new **sumQuantity** column. We finalize the code by using the **ungroup()** function to make sure that there aren't any groups left going forward.

In [15]:
productSales <- df %>%
    select(Item, Quantity) %>%
    group_by(Item) %>%
    summarize(sumQuantity = sum(Quantity)) %>%
    ungroup()

productSales

StatementMeta(, 1b31353d-3791-4ef0-8beb-6d0eee069bdb, 17, Finished, Available)

# Source: spark<?> [?? x 2]
   Item                           sumQuantity
   <chr>                                <dbl>
 1 Mountain-200 Black, 42                 776
 2 Touring-1000 Yellow, 46                148
 3 Touring-1000 Blue, 54                  134
 4 Short-Sleeve Classic Jersey, S         432
 5 Women's Mountain Shorts, S             292
 6 Long-Sleeve Logo Jersey, L             468
 7 Mountain-400-W Silver, 42              118
 8 Racing Socks, M                        330
 9 Mountain-100 Silver, 42                 84
10 Mountain-200 Silver, 42                718
# ℹ more rows
Warning message:
Missing values are always removed in SQL aggregation functions.
Use `na.rm = TRUE` to silence this warning
This warning is displayed once every 8 hours. 

3. Add another new code cell to the notebook, and enter the following code in it:

4. Run the code cell you added, and note that the results show the number of sales orders per year. Note that we're using the **year()** function from [{lubridate}](https://lubridate.tidyverse.org/) inside a **mutate()** function to extract the year component of the OrderDate field and create a new column **Year**. The data is then grouped by the derived Year column and the count of rows in each group is calculated inside the **summarize()** function with **n()** before finally the **arrange()** function is used to sort the resulting dataframe.

In [16]:
yearlySales <- df %>%
    mutate(Year = year(OrderDate)) %>%
    group_by(Year) %>%
    summarize(count = n()) %>%
    ungroup() %>%
    arrange(Year)
    
yearlySales

StatementMeta(, 1b31353d-3791-4ef0-8beb-6d0eee069bdb, 18, Finished, Available)

# Source:     spark<?> [?? x 2]
# Ordered by: Year
   Year count
  <int> <dbl>
1  2019  2401
2  2020  5465
3  2021 57567

## Use Spark to transform data files
A common task for data engineers is to ingest data in a particular format or structure, and transform it for further downstream processing or analysis.

### Use {dplyr}, base R and {SparkR} functions to transform data
1. Add another new code cell to the notebook, and enter the following code in it:

2. Run the code to create a new dataframe from the original order data with the following transformations:
    - Add **Year** and **Month** columns based on the OrderDate column
    - Add **FirstName** and **LastName** columns based on the **CustomerName** column
    - Use **Select()** to filter and reorder the columns, removing the CustomerName column

3. Review the output and verify that the transformations have been made to the data.

In [17]:
## Create Year and Month columns
transformed_df <- df %>%
    mutate(
        Year      = year(OrderDate),
        Month     = month(OrderDate),
        # Since separate() from {tidyr} didn't work, I decided to use split() from base R and element_at() from SparkR
        NameParts = split(CustomerName, " "),
        FirstName = element_at(NameParts, 1L),  
        LastName  = element_at(NameParts, 2L)
  ) %>%
  # Select and reorder wanted columns
  select(
    SalesOrderNumber,
    SalesOrderLineNumber,
    OrderDate,
    Year,
    Month,
    FirstName,
    LastName,
    Email,
    Item,
    Quantity,
    UnitPrice,
    Tax
  )

# Display the first six orders
head(transformed_df)

StatementMeta(, 1b31353d-3791-4ef0-8beb-6d0eee069bdb, 19, Finished, Available)

# Source: spark<?> [?? x 12]
  SalesOrderNumber SalesOrderLineNumber OrderDate   Year Month FirstName
  <chr>                           <int> <date>     <int> <int> <chr>    
1 SO43701                             1 2019-07-01  2019     7 Christy  
2 SO43704                             1 2019-07-01  2019     7 Julio    
3 SO43705                             1 2019-07-01  2019     7 Curtis   
4 SO43700                             1 2019-07-01  2019     7 Ruben    
5 SO43703                             1 2019-07-01  2019     7 Albert   
6 SO43697                             1 2019-07-01  2019     7 Cole     
# ℹ 6 more variables: LastName <chr>, Email <chr>, Item <chr>, Quantity <int>,
#   UnitPrice <dbl>, Tax <dbl>

### Save the transformed data
1. Add a new cell with the following code to save the transformed dataframe in [Parquet](https://parquet.apache.org/) format (overwriting the data if it already exists):

**Note:** Commonly, Parquet format is preferred for data files that you will use for further analysis or ingestion into an analytical store. Parquet is a very efficient format that is supported by most large scale data analytics systems. In fact, sometimes your data transformation requirement may simply be to convert data from another format (such as CSV) to Parquet!

2. Run the cell and wait for the message that the data has been saved. Then, in the **Lakehouses** pane on the left, in the **…** menu for the **Files** node, select **Refresh**; and select the **transformed_data/r** folder to verify that it contains a new folder named **orders**, which in turn contains one or more Parquet files.

In [18]:
transformed_df %>%
    spark_write_parquet(
        path = "Files/transformed_data/r/orders",
        mode = "overwrite"
    )

print ("Transformed data saved!")

StatementMeta(, 1b31353d-3791-4ef0-8beb-6d0eee069bdb, 20, Finished, Available)

[1] "Transformed data saved!"

3. Add a new cell with the following code to load a new dataframe from the parquet files in the **transformed_data/r/orders** folder:

4. Run the cell and verify that the results show the order data that has been loaded from the parquet files.

In [19]:
orders_df <- spark_read_parquet(
    sc,
    path = "Files/transformed_data/r/orders"
)

orders_df

StatementMeta(, 1b31353d-3791-4ef0-8beb-6d0eee069bdb, 21, Finished, Available)

# Source: spark<orders_9f61f001_d1f1_437b_b429_b08dc1ab359b> [?? x 12]
   SalesOrderNumber SalesOrderLineNumber OrderDate   Year Month FirstName
   <chr>                           <int> <date>     <int> <int> <chr>    
 1 SO44067                             1 2019-08-01  2019     8 Krystal  
 2 SO44063                             1 2019-08-01  2019     8 Anne     
 3 SO44064                             1 2019-08-01  2019     8 Richard  
 4 SO44066                             1 2019-08-01  2019     8 Carlos   
 5 SO44065                             1 2019-08-01  2019     8 Xavier   
 6 SO44062                             1 2019-08-01  2019     8 Rebecca  
 7 SO44061                             1 2019-08-01  2019     8 Ryan     
 8 SO44069                             1 2019-08-02  2019     8 Shannon  
 9 SO44070                             1 2019-08-02  2019     8 Lucas    
10 SO44073                             1 2019-08-02  2019     8 Adam     
# ℹ more rows
# ℹ 6 more variables: LastN

## Save data in partitioned files
1. Add a new cell with the following code; which saves the dataframe, partitioning the data by **Year** and **Month**:

2. Run the cell and wait for the message that the data has been saved. Then, in the **Lakehouses** pane on the left, in the **…** menu for the **Files** node, select **Refresh**; and expand the **partitioned_data/r/orders** folder to verify that it contains a hierarchy of folders named **Year=xxxx**, each containing folders named **Month=xxxx**. Each month folder contains a parquet file with the orders for that month.

Partitioning data files is a common way to optimize performance when dealing with large volumes of data. This technique can significant improve performance and make it easier to filter data.

In [20]:
transformed_df %>%
    spark_write_parquet(
        path         = "Files/partitioned_data/r/",
        mode         = "overwrite",
        partition_by = c("Year", "Month")
    )

print ("Transformed data saved!")

StatementMeta(, 1b31353d-3791-4ef0-8beb-6d0eee069bdb, 22, Finished, Available)

[1] "Transformed data saved!"

3. Add a new cell with the following code to load a new dataframe from the **orders.parquet** file:

4. Run the cell and verify that the results show the order data for sales in 2021. Note that the partitioning columns specified in the path (**Year** and **Month**) are not included in the dataframe.

In [21]:
orders_2021_df <- spark_read_parquet(
    sc,
    name = "orders_2021",
    path = "Files/partitioned_data/r/Year=2021/Month=*"
)

orders_2021_df

StatementMeta(, 1b31353d-3791-4ef0-8beb-6d0eee069bdb, 23, Finished, Available)

# Source: spark<orders_2021> [?? x 10]
   SalesOrderNumber SalesOrderLineNumber OrderDate  FirstName LastName Email    
   <chr>                           <int> <date>     <chr>     <chr>    <chr>    
 1 SO59161                             6 2021-11-01 Isabella  Long     isabella…
 2 SO59184                             6 2021-11-01 Destiny   Anderson destiny1…
 3 SO59384                             6 2021-11-04 Miranda   Ross     miranda4…
 4 SO59688                             6 2021-11-08 Mya       Butler   mya14@ad…
 5 SO59789                             6 2021-11-10 Natasha   Carlson  natasha1…
 6 SO59973                             6 2021-11-12 Meagan    Arun     meagan7@…
 7 SO60232                             6 2021-11-16 Kristi    Malhotra kristi21…
 8 SO60233                             6 2021-11-16 Jason     Mitchell jason40@…
 9 SO60233                             7 2021-11-16 Jason     Mitchell jason40@…
10 SO60466                             6 2021-11-20 Colin     Ye      

## Work with tables and SQL
As you’ve seen, the native methods of the dataframe object enable you to query and analyze data from a file quite effectively. However, many data analysts are more comfortable working with tables that they can query using SQL syntax. Spark provides a metastore in which you can define relational tables. The Spark SQL library that provides the dataframe object also supports the use of SQL statements to query tables in the metastore. By using these capabilities of Spark, you can combine the flexibility of a data lake with the structured data schema and SQL-based queries of a relational data warehouse - hence the term “data lakehouse”.

### Create a table
Tables in a Spark metastore are relational abstractions over files in the data lake. tables can be managed (in which case the files are managed by the metastore) or external (in which case the table references a file location in the data lake that you manage independently of the metastore).

1. Add a new code cell to the notebook, and enter the following code, which saves the dataframe of sales order data as a table named **r_salesorders**:

**Note:** It’s worth noting a couple of things about this example. Firstly, no explicit path is provided, so the files for the table will be managed by the metastore. Secondly, the table is saved in **delta** format. You can create tables based on multiple file formats (including CSV, Parquet, Avro, and others) but delta lake is a Spark technology that adds relational database capabilities to tables; including support for transactions, row versioning, and other useful features. Creating tables in delta format is preferred for data lakehouses in Fabric.

2. Run the code cell and review the output, which describes the definition of the new table.

3. In the **Lakehouses** pane, in the **…** menu for the **Tables** folder, select **Refresh**. Then expand the Tables node and verify that the **r_salesorders** table has been created.

In [23]:
# Create a new table
df %>% spark_write_table(
  "r_salesorders",
  mode = "overwrite"
)

# Execute the SQL command
sdf_sql(sc, "DESCRIBE EXTENDED r_salesorders")

StatementMeta(, 1b31353d-3791-4ef0-8beb-6d0eee069bdb, 25, Finished, Available)

# Source: spark<?> [?? x 3]
   col_name               data_type comment
   <chr>                  <chr>     <chr>  
 1 "SalesOrderNumber"     "string"  <NA>   
 2 "SalesOrderLineNumber" "int"     <NA>   
 3 "OrderDate"            "date"    <NA>   
 4 "CustomerName"         "string"  <NA>   
 5 "Email"                "string"  <NA>   
 6 "Item"                 "string"  <NA>   
 7 "Quantity"             "int"     <NA>   
 8 "UnitPrice"            "double"  <NA>   
 9 "Tax"                  "double"  <NA>   
10 ""                     ""        ""     
# ℹ more rows

4. In the **…** menu for the **r_salesorders** table, select **Load data** > **Spark**.

A new code cell containing code similar to the following example is added to the notebook:

5. Run the new code, which uses the Spark SQL library to embed a SQL query against the **r_salesorder** table in {sparklyr} code and load the results of the query into a dataframe.

In [25]:
sdf_sql(sc, "SELECT * FROM dp_600.r_salesorders LIMIT 10")

StatementMeta(, 1b31353d-3791-4ef0-8beb-6d0eee069bdb, 27, Finished, Available)

# Source: spark<?> [?? x 9]
   SalesOrderNumber SalesOrderLineNumber OrderDate  CustomerName     Email Item 
   <chr>                           <int> <date>     <chr>            <chr> <chr>
 1 SO58845                             8 2021-10-30 Peter She        pete… Long…
 2 SO58845                             8 2021-10-30 Peter She        pete… Long…
 3 SO51555                             7 2021-06-23 Chloe Garcia     chlo… Patc…
 4 SO54042                             7 2021-08-09 Logan Collins    loga… Half…
 5 SO54784                             7 2021-08-22 Autumn Li        autu… All-…
 6 SO58572                             7 2021-10-25 Cesar Sara       cesa… Shor…
 7 SO58845                             7 2021-10-30 Peter She        pete… Spor…
 8 SO60233                             7 2021-11-16 Jason Mitchell   jaso… Spor…
 9 SO61412                             7 2021-12-03 Nathaniel Cooper nath… Shor…
10 SO62984                             7 2021-12-29 Miguel Sanchez   migu… Raci…


### Run SQL code in a cell
While it’s useful to be able to embed SQL statements into a cell containing SparkR code, data analysts often just want to work directly in SQL.

1. Add a new code cell to the notebook, and enter the following code in it:

2. Run the cell and review the results.

In [26]:
%%sql
SELECT 
    YEAR(OrderDate) AS OrderYear,
    SUM((UnitPrice * Quantity) + Tax) AS GrossRevenue
FROM 
    r_salesorders
GROUP BY
    YEAR(OrderDate)
ORDER BY
    OrderYear
;

StatementMeta(, 1b31353d-3791-4ef0-8beb-6d0eee069bdb, 28, Finished, Available)

<Spark SQL result set with 3 rows and 2 fields>

Observe that:
- The %%sql line at the beginning of the cell (called a magic) indicates that the Spark SQL language runtime should be used to run the code in this cell instead of SparkR.
- The SQL code references the **r_salesorders** table that you created previously.
- The output from the SQL query is automatically displayed as the result under the cell.

**Note:** For more information about Spark SQL and dataframes, see the [Spark SQL documentation](https://spark.apache.org/docs/2.2.0/sql-programming-guide.html).

## Visualize data with Spark
A picture is proverbially worth a thousand words, and a chart is often better than a thousand rows of data. While notebooks in Fabric include a built in chart view for data that is displayed from a dataframe or Spark SQL query, it is not designed for comprehensive charting. However, you can use R graphics libraries like [{ggplot2}](https://ggplot2.tidyverse.org/) to create charts from data in dataframes.


### View results as a chart
1. Add a new code cell to the notebook, and enter the following code in it:

2. Run the code and observe that it returns the data from the **r_salesorders** view you created previously.
3. In the results section beneath the cell, change the **View** option from **Table** to **Chart**.
4. Use the **Customize chart** button at the top right of the chart to display the options pane for the chart. Then set the options as follows and select **Apply**:
    - **Chart type**: Bar chart
    - **Key**: Item
    - **Values**: Quantity
    - **Series Group**: Leave blank
    - **Aggregation**: Sum
    - **Stacked**: Unselected

In [27]:
%%sql
SELECT * 
FROM r_salesorders

StatementMeta(, 1b31353d-3791-4ef0-8beb-6d0eee069bdb, 29, Finished, Available)

<Spark SQL result set with 1000 rows and 9 fields>

### Get the data for the visualizations

1. Add a new code cell to the notebook, and enter the following code in it:

2. Run the code and observe that it returns a Spark dataframe containing the yearly revenue.

**Note!** I added the row **COUNT(*) AS count** to the SQL query. We'll be needing the counts later on for a visualization.

In [28]:
sqlQuery <- "SELECT 
                CAST(YEAR(OrderDate) AS CHAR(4)) AS OrderYear, \ 
                SUM((UnitPrice * Quantity) + Tax) AS GrossRevenue, \
                COUNT(*) AS count \
            FROM 
                salesorders \
            GROUP BY
                CAST(YEAR(OrderDate) AS CHAR(4)) \
            ORDER BY
                OrderYear
            ;"

df_spark <- sdf_sql(sc, sqlQuery)
df_spark

StatementMeta(, 1b31353d-3791-4ef0-8beb-6d0eee069bdb, 30, Finished, Available)

# Source: spark<?> [?? x 3]
  OrderYear GrossRevenue count
  <chr>            <dbl> <dbl>
1 <NA>               NA      1
2 2019          8344340.  2402
3 2020         13764519.  5466
4 2021         23095671. 57568

### Get started with **{ggplot2}**
To visualize the data as a chart, we’ll start by using the **{ggplot2}** library. This library is the core plotting library on which many others are based, and provides a great deal of flexibility in creating charts.

1. Add a new code cell to the notebook, and add the following code to it:

2. Run the cell and review the results, which consist of a column chart with the total gross revenue for each year. The default settings result in a usable chart, but there’s considerable scope to customize it.

In [29]:
# Create a bar plot of revenue by year
df_spark %>%
    ggplot(aes(OrderYear, GrossRevenue)) +
    geom_bar(stat = "identity")

StatementMeta(, 1b31353d-3791-4ef0-8beb-6d0eee069bdb, 31, Finished, Available)

Warning message:
Removed 1 row containing missing values or values outside the scale range
(`geom_bar()`). 

3. Modify the code to plot the chart as follows:

4. Re-run the code cell and view the results. The chart now includes a little more information.

In [30]:
# Create a bar plot of revenue by year
df_spark %>%
    ggplot(aes(OrderYear, GrossRevenue)) +
    geom_bar(stat = "identity") +
    
    # Customize the chart
    theme_classic() +
    labs(
        title = "Revenue by Year",
        x     = "Year",
        y     = "Revenue"
    )

StatementMeta(, 1b31353d-3791-4ef0-8beb-6d0eee069bdb, 32, Finished, Available)

Warning message:
Removed 1 row containing missing values or values outside the scale range
(`geom_bar()`). 

A figure can contain multiple subplots. There are multiple ways to do it, but I like using the [{patchwork}](https://patchwork.data-imaginist.com/) package.

5. Modify the code to plot the chart as follows:

In [36]:
# Create a bar plot of revenue by year
p1 <- df_spark %>%
    ggplot(aes(OrderYear, GrossRevenue)) +
    geom_bar(stat = "identity") +
    theme_classic() +
    labs(
        title = "Revenue by Year",
        x     = "Year",
        y     = "Revenue"
    )

# Create a pie chart of orders by year
p2 <- df_spark %>%
    filter(!is.na(OrderYear)) %>%
    collect() %>%
    ggpie(
        group_key  = "OrderYear", 
        count_type = "count",
        label_info = "count",
        label_pos  = "out",
        label_type = "horizon"
    ) +
    labs(title = "Orders by Year")

p2

StatementMeta(, 1b31353d-3791-4ef0-8beb-6d0eee069bdb, 38, Finished, Available)

6. Add a new code cell to the notebook, and enter the following code in it:

7. Run the code and observe that it displays a lince chart.

In [33]:
# Create a line chart
df_spark %>%
    filter(!is.na(OrderYear)) %>%
    ggplot(aes(OrderYear, GrossRevenue)) +
    geom_line() +
    theme_classic() +
    labs(
        title = "Revenue by Year",
        x     = "Year",
        y     = "Revenue"
    )

StatementMeta(, 1b31353d-3791-4ef0-8beb-6d0eee069bdb, 35, Finished, Available)

`geom_line()`: Each group consists of only one observation.
ℹ Do you need to adjust the group aesthetic?

## Save the notebook and end the Spark session
Now that you’ve finished working with the data, you can save the notebook with a meaningful name and end the Spark session.

1. In the notebook menu bar, use the **⚙️ Settings** icon to view the notebook settings.
2. Set the **Name** of the notebook to **Sales Order Data Exploration with SparkR**, and then close the settings pane.
3. Run the following code to **disconnect** the Spark connection:

In [37]:
# Disconnect the Spark connection 
spark_disconnect(sc)

StatementMeta(, 1b31353d-3791-4ef0-8beb-6d0eee069bdb, 39, Finished, Available)

## Clean up resources
In this exercise, you’ve learned how to use Spark to work with data in Microsoft Fabric.

If you’ve finished exploring your lakehouse, you can delete the workspace you created for this exercise.

1. In the bar on the left, select the icon for your workspace to view all of the items it contains.
2. In the **…** menu on the toolbar, select **Workspace Settings**.
3. In the **Other** section, select **Remove This Workspace**.